# 🕳️ Pothole Detection Training with YOLOv8

This notebook trains a YOLOv8 model to detect potholes using a dataset from Roboflow/Kaggle.
**Runtime**: Make sure to set Runtime > Change runtime type > Hardware accelerator to **T4 GPU**.

In [ ]:
# 1. Install Ultralytics YOLOv8
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

## 2. Download Dataset
Downloading the 'pothole-detection-2' dataset from Roboflow using your key.

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="7KW7Fth9EOjdB5UTxewS")
project = rf.workspace("pothole-detection-2").project("pothole-detection-2-zvaru")
version = project.version(6)
dataset = version.download("yolov8")

## 3. Train Model
We use `yolov8n.pt` (Nano) for the fastest inference speed on edge/web deployments.

In [ ]:
%cd /content
from ultralytics import YOLO

# Load model
model = YOLO('yolov8n.pt')  # load a pretrained model

# Train
# Find the data.yaml file automatically
import os

data_yaml_path = None
for root, dirs, files in os.walk("/content"):
    if "data.yaml" in files and "pothole" in root.lower():
        data_yaml_path = os.path.join(root, "data.yaml")
        break

# Fallback search if 'pothole' folder name varies
if not data_yaml_path:
     for root, dirs, files in os.walk("/content"):
        if "data.yaml" in files:
            data_yaml_path = os.path.join(root, "data.yaml")
            break

if data_yaml_path:
    print(f"Found data.yaml at: {data_yaml_path}")
    model.train(data=data_yaml_path, epochs=20, imgsz=640)
else:
    print("Could not find data.yaml. Please check the dataset folder manually.")

## 4. Validate and Export
Check how well the model performs and export it for deployment.

In [ ]:
metrics = model.val()  # evaluate model performance on the validation set

In [ ]:
# Download the trained weights
from google.colab import files
try:
  print("Attempting to download best.pt...")
  files.download('/content/runs/detect/train/weights/best.pt')
except:
  print("Automatic download failed. Please manually download: /content/runs/detect/train/weights/best.pt")